# [Cleaning: GICS dataset](#section-title)

In [1]:
#imports
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

---

## Importing and Consolidating the GICS dataset:


- The dataset ```df_g``` was compiled from __[Stock Market MBA](https://stockmarketmba.com/listofstocksinanindustry.php)__ and represents the stock market symbol (ticker), company description, GICS sector, equity type, cap size, market cap (in USD), and average volume of outstanding shares. All data is compiled based on companies' 2022 annual 10-k filings.
- In addition to the data aalyzed below, the Stock Market MBA platform provides additional informaiton on each company including but not limiated to technical indicator charts, valuation metrics, SEC filings by quarter, and press releases.

In [27]:
# Concatenating several csv's showing ticker, makt concentration, and gics class
# Using a list comprehension

dataframes = [pd.read_csv(f"../data/gics_classification/csv/List of Stocks For An Industry ({num}).csv") for num in range(0, 152)]

# Concatenate the list of DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

In [3]:
df.shape

(6987, 9)

In [4]:
# Export these cleaned df's to data folder, for record
df.to_csv("../data/gics.csv", index = False)

In [5]:
# Importing the data frame for GICS info (labeled: g)
df_g = pd.read_csv("../data/gics.csv")

In [6]:
df_g.shape

(6987, 9)

In [7]:
df_g.head(10)

,Symbol,Description,GICS sector,Category1,Category2,Category3,Market Cap,Average Volume,Actions
0,GRP=,Granite Real Estate Inc. Stapled Units Each Co...,Real Estate,US Equity,Units,Mid cap,"$3,765,774,087","2,252",Analyze
1,FEXDR,Fintech Ecosystem Development Corp. Right,Unknown,US Equity,Rights,Micro cap,"$121,803,337","6,395",Analyze
2,HHGCR,HHG Capital Corporation Rights,Unknown,US Equity,Rights,Micro cap,"$78,134,650","7,577",Analyze
3,GRP=,Granite Real Estate Inc. Stapled Units Each Co...,Real Estate,US Equity,Units,Mid cap,"$3,765,774,087","2,252",Analyze
4,FEXDR,Fintech Ecosystem Development Corp. Right,Unknown,US Equity,Rights,Micro cap,"$121,803,337","6,395",Analyze
5,HHGCR,HHG Capital Corporation Rights,Unknown,US Equity,Rights,Micro cap,"$78,134,650","7,577",Analyze
6,CTVA,"Corteva, Inc.",Materials,US Equity,Common stocks,Large cap,"$40,483,113,967","4,083,745",Analyze
7,NTR,Nutrien Ltd.,Materials,US Equity,Common stocks,Large cap,"$36,586,606,605","2,479,812",Analyze
8,FMC,FMC Corp,Materials,US Equity,Common stocks,Large cap,"$14,636,582,004","934,942",Analyze
9,MOS,Mosaic,Materials,US Equity,Common stocks,Large cap,"$14,462,199,011","5,072,753",Analyze


In [8]:
df_g.isnull().sum().sort_values

<bound method Series.sort_values of Symbol            0
Description       0
GICS sector       0
Category1         0
Category2         0
Category3         0
Market Cap        0
Average Volume    0
Actions           0
dtype: int64>

In [9]:
df_g.dtypes

# We need to change the "market cap" and "avergae volume" to integers in the cleaning function below.

Symbol            object
Description       object
GICS sector       object
Category1         object
Category2         object
Category3         object
Market Cap        object
Average Volume    object
Actions           object
dtype: object

In [10]:
# Checking for duplicates within each df

dfs = [df_g] # List created bc I'll eventually add df from fin statement api

for df in dfs:
    print(df["Symbol"].is_unique)
    # A return of false = each ticker is NOT unique = therefore I have duplicates

    print(df["Symbol"].duplicated().any())
    #Different method, same confirmation

False
True


In [11]:
# Drop Duplicates for df_g

print(f' The shape of the GICS dataframe before dropping duplicates is {df_g.shape}')
df_g.drop_duplicates(subset = "Symbol", inplace = True)
print(f' The shape of the GICS dataframe after dropping duplicates is {df_g.shape}')

 The shape of the GICS dataframe before dropping duplicates is (6987, 9)
 The shape of the GICS dataframe after dropping duplicates is (6984, 9)


### Justification for why I dropped duplicates:
By removing duplicates, the shape of the GICS dataframe decreased by 3 rows. 3/6987 reprsents only 0.04% of the data. I felt comfortable that I could drop this percentage without impacting the integrity of the data set at large.

In [12]:
# Cleaning the "Category2" label: 

# First exploring its quantity and type counts
cat2_nunique = df_g["Category2"].nunique()
cat2_value_counts = df_g["Category2"].value_counts()
total_count = df_g.shape[0]

print(f'There are now {total_count} data points after dropping duplicates.\nCategory 2, subsequently relabeled as "Equity Securities", has {cat2_nunique} distinct subcategories which are: \n{cat2_value_counts}')

There are now 6984 data points after dropping duplicates.
Category 2, subsequently relabeled as "Equity Securities", has 9 distinct subcategories which are: 
Common stocks     5455
Units              727
ADRs               438
REITs              168
MLPs                60
BDCs                48
Mortgage REITs      41
Rights              38
Royalty trusts       9
Name: Category2, dtype: int64


In [13]:
# Within real estate sector, how many values are REITS?

print((df_g["GICS sector"] == "Real Estate").sum())
print(df_g[(df_g["GICS sector"] == "Real Estate") & (df_g["Category2"] == "REITs")].shape[0])

237
162


### Justification for which types of Equity Securities were kept and dropped from the dataset:

Within the 9 subcategories for equity securities, we selectively retained data points that allowed for meaningful comparisons of financial metrics. This selection process was predicated on the ability to make like-for-like assessments across income statements (IS), balance sheets (BS), and cash flow statements (CFS), where corresponding line items were present. In certain cases, such as within the context of Real Estate Investment Trusts (REITs), we acknowledged the absence of certain income statement line items like Cost of Goods Sold (COGS), rendering them noncomparable. Despite this discrepancy, the retention of REITs was justified due to the persistence of core top and bottom line similarities, exemplified by Revenue and Net Income.

It's worth noting that though we are including REITS in the EDA stage, their eventual exclusion may be necessary depending on our feature selection for financial metrics. For instance, metrics like gross profit margin (= (Revenue - COGS) / Revenue) would become incongruous because REITS do not specify COGS in their daily operations. Currently, we exercise caution in omitting non-mortgage REITs, particularly since they constitute 162 out of 237 values within the broader real estate sector. Removing their data points could substantially diminish the available sample size within this category.

Furthermore, we refrain from resorting to imputation techniques, as attributing numerical values that lack real-world significance would distort the data's integrity. Our approach prioritizes maintaining the meaningfulness of the financial metrics and their alignment within practical financial contexts.


| Equity Security Type | Description | Quantity | Percent of Data <br>(total: 6984) | Dropped? <br>(Y/N) | Justification <br>for Drop 
| --- | --- | --- | --- | --- | ---
| **Common Stocks** | *Represents partial ownership in a company* | *5455* | *78%* | *N* | *Represent the majority of the data set. <br>The contents of these financial statements <br>are the basis by which we will compare<br>all other equity security types.*
| **Units** | *Represents a portion of a company typically for mutual funds and index funds.* | *727* | *13%* | *Y* | *Dropped because they do not represent individual firms. Units are ownership in a basket of firms, via an investment vehicle like a mutual fund.*
| **ADRs** | *American Depository Receipts: Certificates issued by a US depository bank represeting shares of a foreign company's stock.* | *438* | *6%* | *N* | *Contain comparable financial statements*
| **REITs** | *Real Estate Investment Trusts: Companies that own or finance income-producing real estate across a range of property sectors and use specific valuation metrics like AFFO = Adjusted Funds from Operation, NOI = Net Operating Income, and Cap Rates.* | *168*| *2%* | *N* | *Maintained for the reasons described above.*
| **MLPs** | *Master Limited Partnerships: Common for energy and natural resource sector. MLPs do not pay federal income taxes and all P/L are passed through to individual partners who report on personal income tax returns.* | *60* | *1%* | *N* | *Contain comparable financial statements*
| **BDCs** | *Business Development Companies: Companies who supply capital to private companies. A pass-through entity exempt from<br>taxes at the corproate level.* | *48* | *1%* | *Y* | *Do not contain comparable financial statements<br>and represent a small percentage of data.*
| **Mortgage REITs** | *Mortgage Real Estate Investment Trusts* | *41* | *1%* | *N* | *Maintained for the reasons described above*
| **Rights** | *Financial instrument that grants existing shareholders the option of additional<br>shares at preetermined dates and prices.* | *38* | *1%* | *Y* | *Do not contain comparable financial statements<br>and represent a small percentage of data.*
| **Royalty Trusts** | *Financial instrument that allows investors to receive income from royalties of a particular asset  class, typically related to natural resources.* | *9*| *<1%* | *N* | *Akin to REITS, but for natural resources.*


Sources: 
- __[ADRs](https://www.investopedia.com/terms/a/adr.asp)__
- __[REITs](https://www.reit.com/what-reit?gad=1&gclid=Cj0KCQjw0IGnBhDUARIsAMwFDLlE_su13-1Pvr8qxCEB8OI607EtMzpHZyz206xRF47bJAbTVCxWMAEaAg9zEALw_wcB)__, 
- __[MLPs](https://www.schwab.com/stocks/understand-stocks/mlps#:~:text=Master%20Limited%20Partnerships%20(MLPs)%20are,limited%20partners%20(the%20investors).)__
- __[BDCs](https://www.investopedia.com/terms/b/bdc.asp#:~:text=Key%20Takeaways,and%20some%20capital%20appreciation%20potential.)__
- __[Rights](https://www.investopedia.com/investing/understanding-rights-issues/#:~:text=Defining%20a%20Rights%20Issue,-A%20rights%20issue&text=This%20type%20of%20issue%20gives,stock%20at%20a%20discount%20price.)__
- __[Royalty Trusts](https://www.investopedia.com/terms/r/royaltyincometrust.asp)__

In [14]:
# Analyzing the new shape of my dataframe without the Equity Securities subcategories:
# Note: these subcategories are dropped in the cleaner funciton below.

units_count = (df_g["Category2"] == "Units").sum()
bdc_count = (df_g["Category2"] == "BDCs").sum()
rights_count = (df_g["Category2"] == "Rights").sum()

print(f' The shape of the GICS dataframe before dropping Units, BDCs, and Rights is {df_g.shape}')
rows = df_g.shape[0] - units_count - bdc_count - rights_count
print(f' The shape of the GICS dataframe after dropping Units, BDCs, and Rights is ({rows}, {df_g.shape[1]})')

 The shape of the GICS dataframe before dropping Units, BDCs, and Rights is (6984, 9)
 The shape of the GICS dataframe after dropping Units, BDCs, and Rights is (6171, 9)


### Confirming that the sectors within the data frame match the GICS categorization:

- The Global Industry Classification Standard (GICS) was developed by Morgan Stanley Capital Internation (MSCI) and Standard's & Poor's Financial Services LLC (S&P).
- GICS is a tiered, hierarchical classification system that helps categorize companies within 11 sectors.
- The preprocessing below initially shows that we have 14 sectors, but this is rectified by reformatting text around ```Information Technology``` and realizing that ```Minerals``` is incorrectly categorized and is in fact a subset of ```Materials```.
- There are also 4 values with a gics_sector of ```Unknown```. We reviewed these 4 individually and cross-referenced them with the GICS sectors to replace these sector labels with its correct classifier.

Source: __[GICS](https://www.msci.com/our-solutions/indexes/gics)__

In [15]:
df_g["GICS sector"].nunique()

14

In [16]:
df_g["GICS sector"].unique()

array(['Real Estate', 'Unknown', 'Materials', 'Consumer Staples',
       'Health Care', 'Industrials', 'Consumer Discretionary',
       'Financials', 'Utilities', 'Communication Services',
       'Information Technology', 'Energy', 'Minerals',
       'Information technology'], dtype=object)

In [17]:
df_g[df_g["GICS sector"] == "Unknown"].value_counts()

Symbol  Description                                GICS sector  Category1  Category2      Category3  Market Cap    Average Volume  Actions
ATXG    Addentax Group Corp.                       Unknown      US Equity  Common stocks  Micro cap  $36,160,218   58,920          Analyze    1
FEXDR   Fintech Ecosystem Development Corp. Right  Unknown      US Equity  Rights         Micro cap  $121,803,337  6,395           Analyze    1
HHGCR   HHG Capital Corporation Rights             Unknown      US Equity  Rights         Micro cap  $78,134,650   7,577           Analyze    1
NEOV    NeoVolta Inc                               Unknown      US Equity  Common stocks  Micro cap  $81,206,152   38,915          Analyze    1
dtype: int64

### Creating a cleaner function to clean all columns/rows within each df:

In [18]:
df_g.head()

,Symbol,Description,GICS sector,Category1,Category2,Category3,Market Cap,Average Volume,Actions
0,GRP=,Granite Real Estate Inc. Stapled Units Each Co...,Real Estate,US Equity,Units,Mid cap,"$3,765,774,087","2,252",Analyze
1,FEXDR,Fintech Ecosystem Development Corp. Right,Unknown,US Equity,Rights,Micro cap,"$121,803,337","6,395",Analyze
2,HHGCR,HHG Capital Corporation Rights,Unknown,US Equity,Rights,Micro cap,"$78,134,650","7,577",Analyze
6,CTVA,"Corteva, Inc.",Materials,US Equity,Common stocks,Large cap,"$40,483,113,967","4,083,745",Analyze
7,NTR,Nutrien Ltd.,Materials,US Equity,Common stocks,Large cap,"$36,586,606,605","2,479,812",Analyze


In [19]:
def df_cleaning(df_name):
    
    # Converting columns to lowercase, removing whitespace, removing extraneous symbols
    df_name.columns = df_name.columns.str.replace(' ', '_')
    df_name.columns = df_name.columns.str.lower()
    df_name["symbol"] = df_name["symbol"].str.replace("=", "")
    
    # Reconciling GICS sectors
    df_name["gics_sector"].replace("Information technology", "Information Technology", inplace = True) # Reconciling these columns
    df_name["gics_sector"].replace("Minerals", "Materials", inplace = True) # Minerals is a sector subset of Materials
   
    # Correcting individual gics_sectors data originally labeled as "Unknown"
    df_name.loc[df_name["symbol"] == "ATXG", "gics_sector"] = "Industrials"
    df_name.loc[df_name["symbol"] == "FEXDR", "gics_sector"] = "Financials"
    df_name.loc[df_name["symbol"] == "HHGCR", "gics_sector"] = "Financials"
    df_name.loc[df_name["symbol"] == "NEOV", "gics_sector"] = "Energy"

    # Dropping, Renaming, and Reformating Columns:
    df_name.drop(columns = ["category1"], inplace = True) # Provides no value, all labeled "US Equity"
    df_name.drop(columns = ["market_cap"], inplace = True) # More recent market cap ($) to be pulled from Polygon if needed
    df_name.drop(columns = ["average_volume"], inplace = True) # More recent volume to be pulled from Polygon if needed
    
    df_name.rename(columns={"category2": "equity_securities"}, inplace=True)
    df_name.rename(columns={"category3": "cap_size"}, inplace=True)
    df_name.drop(columns = ["actions"], inplace = True) # Provides no value, all labeled "Analyze"

    # Dropping subcategories within equity_securities column:
    df_g.drop(df_g[df_g["equity_securities"] == "Units"].index, inplace=True) # Drop all "Units"
    df_g.drop(df_g[df_g["equity_securities"] == "BDCs"].index, inplace=True) # Drop all "BDCs"
    df_g.drop(df_g[df_g["equity_securities"] == "Rights"].index, inplace=True) # Drop all "rights"

    return df_name

In [20]:
df_cleaning(df_g)

,symbol,description,gics_sector,equity_securities,cap_size
6,CTVA,"Corteva, Inc.",Materials,Common stocks,Large cap
7,NTR,Nutrien Ltd.,Materials,Common stocks,Large cap
8,FMC,FMC Corp,Materials,Common stocks,Large cap
9,MOS,Mosaic,Materials,Common stocks,Large cap
10,CF,CF Industries Holdings Inc,Materials,Common stocks,Large cap
...,...,...,...,...,...
6982,CLIR,Clearsign Combustion Corp,Information Technology,Common stocks,Micro cap
6983,LIQT,Liqtech International Inc,Industrials,Common stocks,Micro cap
6984,TOMZ,TOMI Environmental Solutions Inc.,Industrials,Common stocks,Micro cap
6985,CLWT,Euro Tech Holdings Company Limited,Industrials,Common stocks,Micro cap


In [21]:
num_sectors = df_g["gics_sector"].nunique()
cat_sectors = df_g["gics_sector"].unique()

print(f'I have now confirmed that there are {num_sectors} unique sectors, wich is consistent with GICS. \nThese sectors are: {cat_sectors}.')

I have now confirmed that there are 11 unique sectors, wich is consistent with GICS. 
These sectors are: ['Materials' 'Consumer Staples' 'Health Care' 'Industrials'
 'Consumer Discretionary' 'Financials' 'Utilities' 'Real Estate'
 'Communication Services' 'Information Technology' 'Energy'].


In [22]:
# Creating a list of all tickers within this data set:
# This will be used in the API below to retrieve fin statements per company

tickers = []
tickers.extend(df_g["symbol"].tolist())
tickers

['CTVA',
 'NTR',
 'FMC',
 'MOS',
 'CF',
 'ICL',
 'SMG',
 'UAN',
 'AVD',
 'BIOX',
 'IPI',
 'MBII',
 'RKDA',
 'CGA',
 'SEED',
 'YTEN',
 'SVFD',
 'RYAAY',
 'DAL',
 'LUV',
 'UAL',
 'AAL',
 'ALK',
 'ZNH',
 'CPA',
 'JBLU',
 'CEA',
 'ULCC',
 'SAVE',
 'ALGT',
 'SNCY',
 'VLRS',
 'SKYW',
 'HA',
 'GOL',
 'MESA',
 'AER',
 'ASR',
 'PAC',
 'OMAB',
 'JOBY',
 'AAWW',
 'CAAP',
 'ASLE',
 'MIC',
 'BLDE',
 'UP',
 'AA',
 'CSTM',
 'ACH',
 'KALU',
 'CENX',
 'VFC',
 'RL',
 'GIL',
 'CPRI',
 'COLM',
 'PVH',
 'UA',
 'UAA',
 'ZGN',
 'KTB',
 'OXM',
 'HBI',
 'LEVI',
 'GOOS',
 'FIGS',
 'SGC',
 'LAKE',
 'VNCE',
 'DLA',
 'JRSH',
 'EVK',
 'XELB',
 'LLL',
 'TJX',
 'LULU',
 'ROST',
 'BURL',
 'ONON',
 'GPS',
 'CRI',
 'AEO',
 'VSCO',
 'URBN',
 'BOOT',
 'BKE',
 'ANF',
 'GES',
 'CHS',
 'GIII',
 'SCVL',
 'DBI',
 'PLCE',
 'GCO',
 'ZUMZ',
 'DXLG',
 'CURV',
 'JILL',
 'DLTH',
 'BIRD',
 'CATO',
 'RENT',
 'TLYS',
 'CTRN',
 'LVLU',
 'EXPR',
 'DBGI',
 'BLK',
 'BX',
 'KKR',
 'BK',
 'APO',
 'AMP',
 'STT',
 'TROW',
 'NTRS',
 'ARES',
 'B

In [26]:
# Create a DataFrame from the list, to save. Then convert back to list in next notebook.
df_tickers_list = pd.DataFrame(tickers, columns=['ticker'])
df.to_csv('../data/cleaned_csvs_interim_steps/tickers_list.csv', index=False)  

In [24]:
df_g.to_csv("../data/cleaned_csvs_interim_steps/gics_cleaned.csv", index = False)

---